In [ ]:
import csv
from io import StringIO
import json
from os import environ
from os.path import isfile, join
import pathlib
from subprocess import Popen

from PIL import Image
import requests

from utils.text import to_slug
from utils.image import resize_img, crop_face, calculate_dominant_color

IMAGES_DIR = join('.', 'imgs')
IMAGES_DIR_RAW = join(IMAGES_DIR, '00_raw')
IMAGES_DIR_HD = join(IMAGES_DIR, '01_hd')
IMAGES_DIR_THUMB = join(IMAGES_DIR, '02_thumb')
IMAGES_DIR_FACES = join(IMAGES_DIR, '00_raw_faces')

WEB_DIR = join('..', 'cordiais-web', 'public')
WEB_DIR_IMAGES = join(WEB_DIR, 'imgs', 'obras')
WEB_DIR_FACES = join(WEB_DIR, 'imgs', 'faces')
WEB_DIR_DATA = join(WEB_DIR, 'data')
WEB_DATA_FILE = join(WEB_DIR_DATA, 'obras.json')

FACE_API_URL = 'https://api-us.faceplusplus.com/facepp/v3/detect'
FACE_API_DATA = {
    'api_key': environ.get('FACEPP_KEY'),
    'api_secret': environ.get('FACEPP_SECRET'),
    'return_attributes': 'emotion,gender,age,facequality,beauty'
}

In [ ]:
with open(WEB_DATA_FILE, 'r') as in_json:
    obras_json = json.load(in_json)


In [ ]:
counts = []
for i in range(10):
    counts.append([])

for slug, v in obras_json.items():
    fcnt = v['faces']
    counts[v['faces']].append(slug)

list(map(len, counts))

In [ ]:
# def get_face_attributes(img_file):
for obra_slug in counts[2]:
    img_file = join(IMAGES_DIR_HD, '%s_%s.jpg' % (obra_slug, 'hd'))
    img = Image.open(img_file).convert('RGB')
    face = {}

    files = {
        'image_file': open(img_file, 'rb')
    }

    res = requests.post(FACE_API_URL, files=files, data=FACE_API_DATA)
    res_o = json.loads(res.text)

    if res.ok:
        if res_o['face_num'] > 1:
            females = sorted(
                [f for f in res_o['faces'][:5] if f['attributes']['gender']['value'] == 'Female'],
                key=lambda x: x['face_rectangle']['width'],
                reverse=True)
            if len(females) > 0:
                top_rect = females[0]['face_rectangle']
                top_image = img.crop((
                    top_rect['left'],
                    top_rect['top'],
                    top_rect['left']+top_rect['width'],
                    top_rect['top']+top_rect['height']))
                print("F: ", obra_slug)
                display(top_image)
            else:
                faces = sorted(
                    [f for f in res_o['faces'][:5]],
                    key=lambda x: x['face_rectangle']['width'],
                    reverse=True)
                top_rect = faces[0]['face_rectangle']
                top_image = img.crop((
                    top_rect['left'],
                    top_rect['top'],
                    top_rect['left']+top_rect['width'],
                    top_rect['top']+top_rect['height']))
                print("?F?: ", obra_slug)
                display(top_image)
